In [1]:
from web3 import Web3
from ens import ENS
# 准备 alchemy API 
# 可以参考https://github.com/AmazingAng/WTFSolidity/blob/main/Topics/Tools/TOOL04_Alchemy/readme.md 
ALCHEMY_MAINNET_URL = 'https://eth-mainnet.g.alchemy.com/v2/oKmOQKbneVkxgHZfibs-iFhIlIAl6HDN'
Provider = Web3(Web3.HTTPProvider(ALCHEMY_MAINNET_URL))
Provider.is_connected()
ns = ENS.from_web3(Provider)

In [2]:
from eth_account import Account
private_key = "0x227dbb8586117d55284e26620bc76534dfbd2394be34cf4a09cb775d593b6f2b"
wallet = Account.from_key(private_key)

In [3]:
# DAI合约地址
AddressDAI = Provider.to_checksum_address("0x6B175474E89094C44Da98b954EedeAC495271d0F")
# DAI合约的abi，只包含后续调用的transfer和balanceOf方法
abiDAI = [
    {
        "inputs": [
            {"internalType": "address", "name": "dst", "type": "address"},
            {"internalType": "uint256", "name": "wad", "type": "uint256"},
        ],
        "name": "transfer",
        "outputs": [{"internalType": "bool", "name": "", "type": "bool"}],
        "stateMutability": "nonpayable",
        "type": "function",
    },
    {
        "inputs": [{"internalType": "address", "name": "", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "stateMutability": "view",
        "type": "function",
    }
]
# 创建DAI合约实例
contractDAI = Provider.eth.contract(address = AddressDAI, abi = abiDAI)

In [4]:
# 获取钱包地址
address = wallet.address
print("1. 读取测试钱包的DAI余额")
balanceDAI = contractDAI.caller.balanceOf(address)
print("DAI的持仓为：", balanceDAI)

1. 读取测试钱包的DAI余额


DAI的持仓为： 0


In [6]:
print("2.  用call尝试调用transfer转账1 DAI，msg.sender为Vitalik地址")
try:
    tx = contractDAI.functions.transfer(ns.address("vitalik.eth"),Provider.to_wei(1,'ether')).call({"from":ns.address("vitalik.eth")})
    print(f"本次交易{'成功' if tx else '失败'}")
except Exception as error:
    print(f"交易失败，失败原因为{error}")

2.  用call尝试调用transfer转账1 DAI，msg.sender为Vitalik地址
本次交易成功


In [7]:
print("3.  用call尝试调用transfer转账10000 DAI，msg.sender为自己的钱包地址")
try:
    tx = contractDAI.functions.transfer(ns.address("vitalik.eth"),Provider.to_wei(10000,'ether')).call({"from":address})
    print(f"本次交易{'成功' if tx else '失败'}")
except Exception as error:
    print(f"交易失败，失败原因为{error}")

3.  用call尝试调用transfer转账10000 DAI，msg.sender为自己的钱包地址
交易失败，失败原因为execution reverted: Dai/insufficient-balance
